## Cards

In [1]:
import os
import json
import uuid
import pandas as pd
import datetime
import shutil
import csv

lookup_path = './lookup/'
source_path = './out/'
mappings_path = './mappings/'
#save_path = '../../../../digipolis-arches-shoku-pkg/source/'
source_file_name = './out/monuments.json'

## Functions

In [2]:
def resource_identifier(source_id):
    return str(uuid.uuid5(uuid.NAMESPACE_DNS, source_id))


def generate_uri(source_id):
    return str(f'https://www.geonames.org/{source_id}')


def coordinates_geo_json(source_value): 
    
    return f'POINT {source_value}'.replace(",", "")


def coordinates_wkt(source_value):    
    return f'{source_value}'.replace(",", "")


def get_constants(node_mapping):
    constant_dict = {}
    constants = node_mapping['constants']
    for constant in constants:
        for constant_label, value in constant.items():
            constant_dict[constant_label] = value
    return constant_dict


def lookup(code, lookup_file):
    lookup_df = pd.read_csv(lookup_path + lookup_file)
    try:
        concept_name = lookup_df[lookup_df['code'] == code]['concept'].iloc[0]
        return concept_name
    except:
        print(f'{code} NOT FOUND IN {lookup_file}')
        return f'{code} NOT FOUND IN {lookup_file}'


## Save csv

In [3]:
def save_mapped_set(source_df, _package, _resource_model, _card):
    
    package = _package.split('_')[1]     
    
    file_name = '%s/%s/%s/%s.csv' % (save_path, package, _resource_model, _card)
    source_df.to_csv(file_name, index=False)    
    del source_df
    return 'Saved OK'

In [4]:
def related_resource(source_value):
    resourceid = str(uuid.uuid5(uuid.NAMESPACE_DNS, source_value))
    resourceXresourceId = str(uuid.uuid5(uuid.NAMESPACE_DNS, source_value + 'relation'))
    relation = '[{"resourceId": "%s", "ontologyProperty": "", "resourceXresourceId": "%s", "inverseOntologyProperty": ""}]' % (resourceid, resourceXresourceId)
    return relation

## Organisation

In [5]:
def make_religious_order(source_list, node_mappings, graph, dataset, card_name, card_order):
    print(card_name)
    obj_dict = {}
    obj_list = []    
    for obj in source_list:

        if 'religiousOrder' in obj.keys():
            if isinstance(obj['religiousOrder'], dict):
                obj_dict['ResourceID'] = str(uuid.uuid5(uuid.NAMESPACE_DNS, obj['religiousOrder']['Concept']['id']))
                obj_dict['card'] = card_name
                obj_dict['card_order'] = '0'
                obj_dict['name_content'] = obj['religiousOrder']['Concept']['prefLabel']
                obj_dict['name_content_type'] = 'Primary'
                obj_dict['name_content_language'] = 'Nederlands'
                obj_dict['identifier_content'] = obj['religiousOrder']['Concept']['id']
                obj_dict['organisation_type'] = 'Religious Order'
                try:
                    obj_dict['alt_names'] = obj['religiousOrder']['Concept']['altLabel']
                    obj_dict['other_identifier_content'] = obj['religiousOrder']['Concept']['wikidata']
                    obj_dict['other_identifier_type'] = '"Brabant Cloud ARK Identifier"'
                except:
                    pass
                obj_list.append(obj_dict)
                obj_dict = {}

            elif isinstance(obj['religiousOrder'], list):                
                for concept in obj['religiousOrder']:                    
                    obj_dict['ResourceID'] = str(uuid.uuid5(uuid.NAMESPACE_DNS, concept['Concept']['id']))
                    obj_dict['card'] = card_name
                    obj_dict['card_order'] = '0'
                    obj_dict['name_content'] = concept['Concept']['prefLabel']
                    obj_dict['name_content_type'] = 'Primary'
                    obj_dict['name_content_language'] = 'Nederlands'
                    obj_dict['identifier_content'] = concept['Concept']['id']
                    obj_dict['name_content_type'] = 'Nederlands'
                    
                    obj_dict['organisation_type'] = 'Religious Order'                    
                    try:
                        obj_dict['alt_names'] = concept['Concept']['altLabel']
                        obj_dict['other_identifier_content'] = concept['Concept']['wikidata']
                        obj_dict['other_identifier_type'] = "Brabant Cloud ARK Identifier"
                    except:
                        pass
                    
                    obj_list.append(obj_dict)
                    obj_dict = {}

    source_df = pd.DataFrame(obj_list)
    source_df = source_df.drop_duplicates(subset=['ResourceID'])
    number_of_records = source_df.shape[0]
    unique_list = source_df.to_dict('records')
    if number_of_records > 0:                       
        #save_it = save_mapped_set(source_df, package, resource_model, card)
        source_df.to_csv(f'out/organisation/organisation_religious_order.csv', index =False)
        #return '%s saved: %s' % (card_name, number_of_records)
        return unique_list

## Alternate names

In [6]:
def make_alternate_names(source_list, node_mappings, graph, dataset, card_name, card_order):
    
    obj_dict = {}
    obj_list = []    
    for obj in source_list:
        if 'alt_names' in obj.keys():            
            if isinstance(obj['alt_names'], dict):  
                obj_dict['ResourceID'] = obj['ResourceID']
                obj_dict['id'] = obj['other_identifier_content']
                obj_dict['card'] = card_name
                obj_dict['card_order'] = '1'
                obj_dict['alternative_name_content'] = alt_label
                obj_dict['alternative_name_content_type'] = 'Alternative'
                obj_dict['alternative_name_content_language'] = 'Nederlands'
                          
                obj_list.append(obj_dict)
                obj_dict = {}
            
            elif isinstance(obj['alt_names'], list): 
                for alt_name in obj['alt_names']:
                    obj_dict['ResourceID'] = obj['ResourceID']
                    obj_dict['id'] = obj['other_identifier_content']
                    obj_dict['card'] = card_name
                    obj_dict['card_order'] = '1'
                    obj_dict['alternative_name_content'] = alt_name
                    obj_dict['alternative_name_content_type'] = 'Alternative'
                    obj_dict['alternative_name_content_language'] = 'Nederlands'
                    
                    obj_list.append(obj_dict)
                    obj_dict = {}
        
        
    source_df = pd.DataFrame(obj_list)
    number_of_records = len(obj_list)
        
    if number_of_records > 0:                       
        #save_it = save_mapped_set(source_df, package, resource_model, card)
        source_df.to_csv(f'out/organisation/organisation_alternate_names.csv', index =False)
        #return '%s saved: %s' % (card_name, number_of_records)
        return obj_list

## Generate dataset

In [7]:
%%time

with open(mappings_path + 'mappings_organisation.json') as f:
    mappings_dict = json.load(f)  
    
    record_dict = {}
    record_list = []

    single_list = []
    religious_order_list = []
    alternate_names_list = []

    name = mappings_dict['name']
    graph = mappings_dict['graph']
    print('-', name)
    print('  -', graph)

    with open(source_file_name) as f:
        records = json.load(f)
        _landmarks = records['records']['record']

    for cards, cards_values in mappings_dict['mappings'].items():  
        card_name = cards
        
        for item,item_value in cards_values.items():
            if item == 'dataset': 
                dataset = item_value
            if item == 'card_order':
                card_order = item_value
            if item == 'filter':
                card_filter = item_value    
            if item == 'node_mappings':
                node_mappings = item_value    
                
        node_mappings_keys = [item['from'] for item in node_mappings]
        
        for _record in _landmarks:
            
            record = _record['LandmarksOrHistoricalBuildings']
            record_dict['ResourceID'] = str(uuid.uuid5(uuid.NAMESPACE_DNS, record['id']))
            record_dict['card'] = card_name
            record_dict['card_order'] = card_order

            for mapping_key in node_mappings_keys:
                if mapping_key in record.keys():
                    if len(record[mapping_key]) > 0:                                
                        record_dict[mapping_key] = record[mapping_key]
                    
            record_list.append(record_dict)                            
            record_dict = {}
                        

        if card_name == 'religious_order':
            religious_order_map = node_mappings
            religious_order_list.extend(record_list)                    

        if card_name == 'alternate_names':
            alternate_names_map = node_mappings
            #print(card_name, alternate_names_map)
            alternate_names_list.extend(record_list)                    

        record_list = []
            
            
    
    if len(religious_order_list) > 0:  
        Religiousorder = make_religious_order(religious_order_list, religious_order_map, graph, dataset, 'religious_order', card_order)
        print('    -', "Religiousorder", len(Religiousorder))

    if len(alternate_names_list) > 0:        
        AlternateName = make_alternate_names(Religiousorder, alternate_names_map, graph, dataset, card_name, card_order)
        print('    -', "AlternateName", len(AlternateName))




print('---------') 
print(datetime.datetime.now())
#print(json.dumps(Religiousorder, indent=2))
#%reset -f 

- brabantse_gebouwen
  - Building
religious_order
    - Religiousorder 141
    - AlternateName 983
---------
2025-01-14 15:07:29.650092
CPU times: user 43.8 ms, sys: 10.1 ms, total: 53.9 ms
Wall time: 55.8 ms
